# bibliotecas

In [1]:
import numpy as np
import random
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MaxAbsScaler

# lendo a iris.csv

In [77]:
df = pd.read_csv('iris.csv', nrows=150)

# transformando as saídas categoricas em inteiras

In [78]:
item_encoder = LabelEncoder()
df['variety'] = item_encoder.fit_transform(df.variety)

# pegando as classes da iris e colcoando em y

In [79]:
y = df.variety
df.drop(['variety'],axis=1, inplace=True)

# inserindo vetor de 1´s no dataset (perceptron tem disso)

In [80]:
df.insert(0, "new", np.ones(len(df))*(-1), True) 

# normalizando o dataset

In [81]:
p=MaxAbsScaler()
p.fit(df)
df_max_abs = p.transform(df)
df = pd.DataFrame(data=df_max_abs, columns=df.columns)

# separando em treino e teste

In [82]:
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.20, random_state=42)

# parametros do AG

In [113]:
CHANCE_MUT = .20     # Chance de mutação de um peso qualquer
CHANCE_CO = .25      # Chance de crossing over de um peso qualquer
NUM_INDIVIDUOS = 50  # Tamanho da população
NUM_MELHORES = 3     # Número de indivíduos que são mantidos de uma geração para a próxima

# criando população aleatória

In [84]:
def populacao_aleatoria(n, tamanho):
    """
    Argumentos da Função:
        n: Número de indivíduos
    Saída:
        Uma população aleatória. População é uma lista de indivíduos,
        e cada indivíduo é uma matriz 1x5 de pesos (números).
        Os indivíduos podem tomar valores entre -1 e 1 e cada linha da matriz
        contém os pesos associados.
    """
    populacao = []
    for i in range(n):
        populacao.append(np.random.uniform(-1, 1, tamanho))
    return populacao

# função de aptidão

In [85]:
def funcao_aptidao(individuos, treino, y_treino):
    """
    Argumentos da Função:
        individuos: matriz nxlen(treino[0]) com os pesos do indivíduo.
        treino: amostras de treino.
        y_treino: saídas reais
    Saída:
        Uma lista com o somatório de todos os erros de cada indivíduo e o conjunto de treino.
    """
    #print(individuos.shape, treino.shape, treino.shape)
    #print('********************************************************')
    erros = [(treino @ individuo) - y_treino for individuo in individuos]  
    soma_erros = [sum((erro**2)/len(treino)) for erro in erros] 
    
    return soma_erros

# função de mutação

In [86]:
def mutacao(individuo):
    """
    Argumentos da Função:
        individuo: matriz nxlen(treino[0]) com os pesos do indivíduo.
    Saída:
        Essa função não tem saída. Ela apenas modifica os pesos do indivíduo,
        com chance CHANCE_MUT para cada peso.
    """

    #for i in range(1):
    for j in range(5):
        if np.random.uniform(0, 1) < CHANCE_MUT:
            individuo[j] *= np.random.uniform(-10.0, 10.0) #pode mudar para valores entre 10 e -10

# cruzamento crossover

In [87]:
def crossover(individuo1, individuo2, tamanho):
    """
    Argumentos da Função:
        individuoX: matriz nxlen(treino[0]) com os pesos do individuoX.
    Saída:
        Um novo indivíduo com pesos que podem vir do `individuo1`
        (com chance 1-CHANCE_CO) ou do `individuo2` (com chance CHANCE_CO),
        ou seja, é um cruzamento entre os dois indivíduos. Você também pode pensar
        que essa função cria uma cópia do `individuo1`, mas com chance CHANCE_CO,
        copia os respectivos pesos do `individuo2`.
    """
    filho = individuo1.copy()
    #for i in range(1):
    for j in range(tamanho):
        if np.random.uniform(0, 1) < CHANCE_CO:
            filho[j] = individuo2[j]
    return filho

# função para ordenar a lista de erros obtidas na função de aptidão

In [88]:
def ordenar_lista(lista, ordenacao, decrescente=False):
    """
    Argumentos da Função:
        lista: lista de números a ser ordenada.
        ordenacao: lista auxiliar de números que define a prioridade da
        ordenação.
        decrescente: variável booleana para definir se a lista `ordenacao`
        deve ser ordenada em ordem crescente ou decrescente.
    Saída:
        Uma lista com o conteúdo de `lista` ordenada com base em `ordenacao`.
    Por exemplo,
        ordenar_lista(['a', 'b', 'c', 'd'], [7, 2, 5, 4])
        # retorna ['a', 'c', 'd', 'b'] (o maior número é 7, que corresponde à primeira letra: 'a')
        ordenar_lista(['w', 'x', 'y', 'z'], [3, 8, 2, 1])
        # retorna ['x', 'w', 'y', 'z'] (o maior número é 8, que corresponde à segunda letra: 'x')
    """
    return [x for _, x in sorted(zip(ordenacao, lista), key=lambda p: p[0], reverse=decrescente)]

# função que retorna a próxima geração

In [89]:
def proxima_geracao(populacao, fitness):
    """
    Argumentos da Função:
        populacao: lista de indivíduos.
        fitness: lista de fitness, uma para cada indivíduo.
    Saída:
        A próxima geração com base na população atual.
        Para criar a próxima geração, segue-se o seguinte algoritmo:
          1. Colocar os melhores indivíduos da geração atual na próxima geração.
          2. Até que a população esteja completa:
             2.1. Escolher aleatoriamente dois indivíduos da geração atual.
             2.2. Criar um novo indivíduo a partir desses dois indivíduos usando
                  crossing over.
             2.3. Mutar esse indivíduo.
             2.4. Adicionar esse indivíduo na próxima geração
    """
    # Adicionar os melhores indivíduos da geração atual na próxima geração
    ordenados = ordenar_lista(populacao, fitness)
    proxima_ger = ordenados[:NUM_MELHORES]

    while len(proxima_ger) < NUM_INDIVIDUOS:
        # Você pode usar a função random.choices(populacao, weights=None, k=2) para selecionar `k`
        # elementos aleatórios da população.
        #
        # Se vc passar o argumento `weights`, os indivíduos serão escolhidos com base nos pesos
        # especificados (elementos com pesos maiores são escolhidos mais frequentemente).
        # Uma ideia seria usar o fitness como peso.
        ind1, ind2 = random.choices(populacao, k=2)
        filho = crossover(ind1, ind2, len(populacao[0]))
        mutacao(filho)
        proxima_ger.append(filho)

    return proxima_ger

In [114]:
# OBS: Todos os prints abaixo são opcionais.
# Eles estão aqui para facilitar a visualização do algoritmo.

num_geracoes = 1000

# Crie a população usando populacao_aleatoria(NUM_INDIVIDUOS)
populacao = populacao_aleatoria(NUM_INDIVIDUOS, len(X_train.values[0]))

for ger in range(num_geracoes):
    
    # Atualize a população usando a função próxima_geração.
    populacao = proxima_geracao(populacao, funcao_aptidao(populacao, X_train, y_train))
    
# Mostre o melhor indivíduo
ordenados = ordenar_lista(populacao, funcao_aptidao(populacao, X_train, y_train))
melhor = ordenados[0]
print('Melhor individuo:', melhor)

Melhor individuo: [ 0.41089114  0.26053792 -0.15112916  0.66782797  2.01088366]


In [115]:
funcao_aptidao(ordenados, X_test, y_test)

[0.033520646802851,
 0.033520646802851,
 0.033520646802851,
 1.3460122405574662,
 139.53896215117115,
 625.4035946969,
 712.557497626891,
 911.1488079659197,
 2428.536947816807,
 5393.850762185462,
 6807.82154678138,
 8647.841937664003,
 9461.644606667352,
 18018.29579036776,
 46232.01370047473,
 54962.751575148475,
 177880.07653695258,
 445249.2064049545,
 536119.5602363285,
 613436.0787303682,
 825318.3543779579,
 1092570.9030841668,
 1935185.201320002,
 1758112.6296456105,
 2527397.8440546994,
 3247140.5321258507,
 3450146.0500247795,
 4119963.3148977323,
 6098688.365457094,
 6175856.084425094,
 7236033.745775566,
 8385201.328598194,
 9376385.73164151,
 14566066.264845556,
 13973465.727119708,
 13321087.482647952,
 20074046.15327731,
 21431469.23654698,
 54807564.57762827,
 86540766.11971669,
 118495679.2186108,
 246593408.2947418,
 333288989.3072517,
 391415193.7137166,
 954313695.9271855,
 1783639610.380061,
 32030656509.299217,
 43207780483425.92,
 43207801867105.8,
 7.9708485546

# saídas obtidas

In [116]:
a = [int(x @ melhor) for x in X_test.values]
np.array(a).reshape(1, len(a))

array([[1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 1, 0, 0, 0, 0, 1, 2, 0, 1, 2, 0, 1,
        0, 1, 1, 2, 1, 2, 0, 0]])

# saída original

In [117]:
b = y_test.values.reshape(1, len(y_test))
b

array([[1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2,
        0, 2, 2, 2, 2, 2, 0, 0]])

# taxa de acerto

In [118]:
 1-abs((a - b).sum()/len(y_test))

0.8

In [ ]:
https://towardsdatascience.com/illustrated-self-attention-2d627e33b20a